In [1]:
import sys
sys.path.insert(0, "/home/GAFOL/work/kspec_gfa_controller/src")  # <-- add *src*, not the package folder

from kspec_gfa_controller.gfa_actions import GFAActions



In [2]:
actions = GFAActions()

2026-01-20 02:24:28,365 [INFO] Initialized GFAEnvironment with role 'plate' and cameras [1, 2, 3, 4, 5, 6]
2026-01-20 02:24:28,365 [INFO] Initializing GFAController with provided config.
2026-01-20 02:24:28,366 [INFO] GFAController initialization complete.
2026-01-20 02:24:28,366 [INFO] Initializing gfa_astrometry class.
2026-01-20 02:24:28,366 [INFO] Initializing GFAGuider...
2026-01-20 02:24:28,367 [DEBUG] Processed dir: /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/procimg
2026-01-20 02:24:28,367 [DEBUG] Final astro dir: /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/astroimg/
2026-01-20 02:24:28,367 [DEBUG] Cutout dir: /home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/cutout
2026-01-20 02:24:28,368 [INFO] GFAGuider setup complete.


In [3]:
await actions.pointing(ra="02:25:00.77", dec="-53:40:06.28", ExpTime=3) # RA, DEC 

2026-01-20 02:24:32,277 [INFO] Starting pointing sequence...
2026-01-20 02:24:32,277 [INFO] Target RA/DEC: 02:25:00.77, -53:40:06.28
2026-01-20 02:24:32,379 [INFO] Solving astrometry for CRVALs using 6 dark-subtracted images (max_workers=4)...
2026-01-20 02:24:32,380 [INFO] Start get_crvals_from_images: n=6 max_workers=4 keep_work_dir=False
2026-01-20 02:24:32,380 [DEBUG] Image list:
/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/pointing_raw_dark/2026-01-20/D20260119_T141119_40103667_exp3s_darksub.fits
/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/pointing_raw_dark/2026-01-20/D20260119_T141119_40103831_exp3s_darksub.fits
/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/pointing_raw_dark/2026-01-20/D20260119_T141120_40103651_exp3s_darksub.fits
/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/pointing_raw_dark/2026-01-20/D20260119_T141120_40103663_exp3s_darksub.fits
/home/GAFOL/work/kspec_gfa_controller/src/ksp

{'status': 'success',
 'message': 'Pointing completed. Computed CRVALs for 6 images.',
 'images': ['D20260119_T141119_40103667_exp3s_darksub.fits',
  'D20260119_T141119_40103831_exp3s_darksub.fits',
  'D20260119_T141120_40103651_exp3s_darksub.fits',
  'D20260119_T141120_40103663_exp3s_darksub.fits',
  'D20260119_T141120_40103833_exp3s_darksub.fits',
  'D20260119_T141120_40103834_exp3s_darksub.fits'],
 'crval1': [nan, nan, nan, nan, nan, nan],
 'crval2': [nan, nan, nan, nan, nan, nan],
 'dark_output_dir': '/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/pointing_raw_dark/2026-01-20'}

In [ ]:
await actions.grab(CamNum=0, ExpTime=0.1, ra="06:57:31.68", dec="-29:06:02.30") # RA, DEC 
#actions.cam_params()

In [ ]:
#await actions.pointing(ra="02:25:00.77", dec="-53:40:06.28", ExpTime=11) # RA, DEC 

In [ ]:
from astropy.io import fits

im = fits.getdata('/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/grab/2026-01-19/D20260119_T141119_40103667_exp3s.fits')

In [ ]:
from photutils.detection import DAOStarFinder

ff = DAOStarFinder(threshold=5e2, fwhm=20)
tb = ff.find_stars(im)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,6))

ax.imshow(np.log10(im), cmap='gray', origin='lower')

for i in range(len(tb)):
    c1 = plt.Circle((tb['xcentroid'][i], tb['ycentroid'][i]), 20, ec="cyan", fc="none")
    ax.add_artist(c1)

In [ ]:
from astropy.io import fits
from astropy.table import Table
import numpy as np

def write_xyls(x, y, out_xyls="stars.xyls"):
    """
    x, y: list/np.ndarray of same length (float/int OK)
    out_xyls: output FITS binary table filename (usually .xyls)
    """
    x = np.asarray(x, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)
    if x.shape != y.shape:
        raise ValueError(f"x and y must have same shape, got {x.shape} vs {y.shape}")

    tbl = Table()
    tbl["X"] = x
    tbl["Y"] = y

    hdu = fits.BinTableHDU(tbl)
    hdu.writeto(out_xyls, overwrite=True)
    print(f"Saved {len(x)} sources -> {out_xyls}")

# 예시 1) x_list, y_list가 있을 때
# write_xyls(x_list, y_list, "stars.xyls")

# 예시 2) points=[(x0,y0),(x1,y1),...] 형태일 때
def write_xyls_from_points(points, out_xyls="stars.xyls"):
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    write_xyls(x, y, out_xyls)


In [ ]:
write_xyls_from_points(tb)

In [ ]:
from astropy.io import fits
from astropy.table import Table
import numpy as np

def write_xyls(x, y, flux, out_xyls="stars.xyls"):
    """
    x, y, flux: list/np.ndarray of same length
    out_xyls: output FITS binary table filename (.xyls)
    """
    x = np.asarray(x, dtype=np.float32)
    y = np.asarray(y, dtype=np.float32)
    flux = np.asarray(flux, dtype=np.float32)

    if not (x.shape == y.shape == flux.shape):
        raise ValueError(
            f"x, y, flux must have same shape, "
            f"got {x.shape}, {y.shape}, {flux.shape}"
        )

    tbl = Table()
    tbl["X"] = x
    tbl["Y"] = y
    tbl["FLUX"] = flux

    hdu = fits.BinTableHDU(tbl)
    hdu.writeto(out_xyls, overwrite=True)

    print(f"Saved {len(x)} sources with flux -> {out_xyls}")

def write_xyls_from_points(points, out_xyls="stars.xyls"):
    """
    points = [(x0, y0, flux0), (x1, y1, flux1), ...]
    """
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    flux = [p[2] for p in points]

    write_xyls(x, y, flux, out_xyls)

In [ ]:
write_xyls_from_points(tb)

In [ ]:
from astropy.io import fits
import astroscrappy
import numpy as np

# 입력 파일
in_fits = "/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/grab/2026-01-19/D20260119_T141120_40103651_exp3s.fits"
out_fits = "D20260119_T141120_40103651_exp3s_lacosmic.fits"

# FITS 읽기
data, header = fits.getdata(in_fits, header=True)
data = data.astype(np.float32)

# L.A.Cosmic 실행
mask, clean = astroscrappy.detect_cosmics(
    data,
    sigclip=5.0,      # detection threshold (보통 4~6)
    sigfrac=0.3,
    objlim=5.0,
    cleantype="median"
)

# 결과 저장
fits.writeto(out_fits, clean, header, overwrite=True)

print(f"Saved cosmic-cleaned image -> {out_fits}")
print(f"Detected cosmic pixels: {mask.sum()}")


In [ ]:
from astropy.io import fits
import astroscrappy
import numpy as np
import matplotlib.pyplot as plt

# 입력 FITS
fits_path = "/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/grab/2026-01-19/D20260119_T141120_40103651_exp3s.fits"

# FITS 읽기
data, header = fits.getdata(fits_path, header=True)
data = data.astype(np.float32)

# L.A.Cosmic 실행
mask, clean = astroscrappy.detect_cosmics(
    data,
    sigclip=5.0,
    sigfrac=0.3,
    objlim=5.0,
    cleantype="median"
)

print(f"Detected cosmic pixels: {mask.sum()}")

# --------- 시각화 ---------
vmin, vmax = np.percentile(data, (5, 99.5))

plt.figure(figsize=(15, 5))

# 원본
plt.subplot(1, 3, 1)
plt.imshow(data, origin="lower", cmap="gray", vmin=vmin, vmax=vmax)
plt.title("Original")
plt.colorbar(shrink=0.7)

# Cosmic 마스크
plt.subplot(1, 3, 2)
plt.imshow(mask, origin="lower", cmap="inferno")
plt.title("Cosmic Mask")
plt.colorbar(shrink=0.7)

# 클린 이미지
plt.subplot(1, 3, 3)
plt.imshow(clean, origin="lower", cmap="gray", vmin=vmin, vmax=vmax)
plt.title("Cleaned (L.A.Cosmic)")
plt.colorbar(shrink=0.7)

plt.tight_layout()
plt.show()


In [19]:
from astropy.io import fits
import numpy as np

test = "/home/GAFOL/work/kspec_gfa_controller/src/kspec_gfa_controller/img/pointing_raw/2026-01-19/D20260119_T141120_40103834_exp3s.fits"
# FITS 읽기
data, header = fits.getdata(test, header=True)
data = data.astype(np.float32)

In [20]:
from scipy.ndimage import maximum_filter, median_filter

def hot_pixel_removal_median_ratio(
    img: np.ndarray,
    factor: float = 5.0,            # 주변 median의 몇 배 이상이면 제거할지
    n_iter: int = 1,
    mode: str = "mirror",
    saturated_value: int | float | None = None,
    eps: float = 1e-6,              # median이 0 근처일 때 0나눔/과잉검출 방지
    abs_threshold: float | None = None,  # (선택) |P - median|이 이것보다 커야 제거
    keep_dtype: bool = True,
):
    """
    주변 8픽셀 median을 기준으로:
      P > median_neighbors * factor  이면 핫픽셀로 보고,
    해당 픽셀을 median_neighbors로 치환.

    abs_threshold를 같이 쓰면 (P - median) 절대 차이도 커야 제거되므로
    어두운 배경에서 과잉 검출되는 것을 줄일 수 있습니다.
    """
    img = np.asarray(img)
    work = img.astype(np.float32, copy=True)

    # 중앙 제외한 8-neighbor footprint
    footprint = np.array([[1, 1, 1],
                          [1, 0, 1],
                          [1, 1, 1]], dtype=np.uint8)

    for _ in range(max(1, int(n_iter))):
        med_nb = median_filter(work, footprint=footprint, mode=mode)

        # ratio 조건: P > median * factor
        denom = np.maximum(np.abs(med_nb), eps)   # median이 0일 때 폭주 방지
        mask = work > denom * float(factor)

        # (선택) 절대 차이 조건도 추가
        if abs_threshold is not None:
            mask &= (work - med_nb) > float(abs_threshold)

        # saturation 값은 제외하고 싶다면
        if saturated_value is not None:
            mask &= (work != float(saturated_value))

        # 치환
        work[mask] = med_nb[mask]

    # dtype 복구
    if keep_dtype:
        if np.issubdtype(img.dtype, np.integer):
            info = np.iinfo(img.dtype)
            work = np.clip(np.rint(work), info.min, info.max).astype(img.dtype)
        else:
            work = work.astype(img.dtype)

    return work

cleaned = hot_pixel_removal_median_ratio(data, factor=1.5, n_iter=2)

from astropy.io import fits

hdr = fits.Header()
hdr['Gain'] = 0
hdr['offset'] = 0
hdr['texp'] = 0

empty_primary = fits.PrimaryHDU(header=hdr, data=cleaned)
empty_primary.writeto(f'hotpix_test_1.5median.fits', overwrite=True)

In [ ]:
#solve-field ./clean/D20260119_T
#133618_40103834_exp1s_hotpix_1p5med.fits   -D /tmp/gfa_xyls_solve   -O --no-plots --no-verify --crpix-center   -X X -Y Y -s FLUX   --scale-units degwidth -L 0.12 -H 0.22   --ra "06:57:31.68" --dec "-29:06:02.30" --radius 2   -l 120  -c 0.1 -E 2